In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2
import os
from zipfile import ZipFile
import time
from datetime import datetime
import itertools

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras import utils
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
train_ds = pd.read_csv("train.csv")
test_ds = pd.read_csv("val.csv")

In [ ]:
num_classes = 6
def class_labels(age):

    if  age == 0:
        return 0
    elif  age == 1:
        return 0
    elif  age == 2:
        return 1
    elif  age == 3:
        return 1
    elif  age == 4:
        return 2
    elif age == 5:
        return 2
    elif  age == 6:
        return 3
    elif  age == 7:
        return 3
    elif  age == 8:
        return 4
    elif  age == 9:
        return 4
    elif  age == 10:
        return 5
    else:
        return 5

In [ ]:
train_ds['target'] = train_ds['age'].map(class_labels)
test_ds['target'] = test_ds['age'].map(class_labels)

In [6]:
import py7zr

archive1 = 'data/eval.7z'
archive2 = 'data/train.7z'

with py7zr.SevenZipFile(archive1, mode='r') as z:
    z.extractall()
    
with py7zr.SevenZipFile(archive2, mode='r') as a:
    a.extractall()

KeyboardInterrupt: 

In [ ]:
# Создание списка имен файлов

train_path = list(train_ds['filename'])
train_ll = list(train_ds['target'])

test_path = list(test_ds['filename'])
test_ll = list(test_ds['target'])

# Преобразование в константы

train_path_tf = tf.constant(train_path)
train_ll_tf = tf.constant(train_ll)

test_path_tf = tf.constant(test_path)
test_ll_tf = tf.constant(test_ll)

In [ ]:
# Функция чтение и преобразование изображений

def _parse_function(filename, label):
    
    image_string = tf.io.read_file(filename)
    image_decoded = tf.io.decode_jpeg(image_string, channels=1)
    label = tf.one_hot(label, num_classes)
    print(image_decoded)

    return image_decoded, label

In [ ]:
# создание дата сета для нейросети

train_dataset = tf.data.Dataset.from_tensor_slices((train_path_tf, train_ll_tf))
train_dataset = train_dataset.map(_parse_function)

train_dataset = train_dataset.batch(32)

test_dataset = tf.data.Dataset.from_tensor_slices((test_path_tf, test_ll_tf))
test_dataset = test_dataset.map(_parse_function)

test_dataset = test_dataset.batch(32)  

In [ ]:
my_cnn_arh = Sequential()

my_cnn_arh.add(Conv2D(filters=32, kernel_size=5, activation='relu', input_shape=(200, 200, 1)))
my_cnn_arh.add(AveragePooling2D(pool_size=(3,3)))

my_cnn_arh.add(Conv2D(filters=128, kernel_size=5, activation='relu'))
my_cnn_arh.add(AveragePooling2D(pool_size=(2,2)))

my_cnn_arh.add(Conv2D(filters=256, kernel_size=3, activation='relu'))
my_cnn_arh.add(AveragePooling2D(pool_size=(2,2)))

my_cnn_arh.add(Conv2D(filters=256, kernel_size=3, activation='relu'))
my_cnn_arh.add(AveragePooling2D(pool_size=(2,2)))

my_cnn_arh.add(Conv2D(filters=192, kernel_size=3, activation='relu'))
my_cnn_arh.add(AveragePooling2D(pool_size=(2,2)))

my_cnn_arh.add(GlobalAveragePooling2D())

my_cnn_arh.add(Dense(512, activation='relu'))
my_cnn_arh.add(Dense(132, activation='relu'))

my_cnn_arh.add(Dense(6, activation='softmax'))

my_cnn_arh.summary()

In [ ]:
my_cnn_arh.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
checkpoint_path = 'save/weight.h5'
checkpoint = ModelCheckpoint(filepath=checkpoint_path, monitor='val_accuracy',save_best_only=True,save_weights_only=False,verbose=1)

In [ ]:
my_cnn_arh_history = my_cnn_arh.fit(train_dataset,validation_data=test_dataset,batch_size=32,epochs=1,callbacks=[ checkpoint],shuffle=False)

In [ ]:
train_loss = my_cnn_arh_history.history['loss']
test_loss = my_cnn_arh_history.history['val_loss']
train_accuracy = my_cnn_arh_history.history['accuracy']
test_accuracy = my_cnn_arh_history.history['val_accuracy']

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(15,7))

ax = ax.ravel()

ax[0].plot(train_loss, label='Train Loss', color='royalblue', marker='o', markersize=5)
ax[0].plot(test_loss, label='Test Loss', color = 'orangered', marker='o', markersize=5)

ax[0].set_xlabel('Epochs', fontsize=14)
ax[0].set_ylabel('Categorical Crossentropy', fontsize=14)

ax[0].legend(fontsize=14)
ax[0].tick_params(axis='both', labelsize=12)

ax[1].plot(train_accuracy, label='Train Accuracy', color='royalblue', marker='o', markersize=5)
ax[1].plot(test_accuracy, label='Test Accuracy', color='orangered', marker='o', markersize=5)

ax[1].set_xlabel('Epochs', fontsize=14)
ax[1].set_ylabel('Accuracy', fontsize=14)

ax[1].legend(fontsize=14)
ax[1].tick_params(axis='both', labelsize=12)

fig.suptitle(x=0.5, y=0.92, t="Линейные графики, показывающие потери и точность модели CNN по эпохам", fontsize=16)

In [ ]:
my_cnn_arh_score = my_cnn_arh.evaluate(test_dataset, verbose=1)

In [ ]:
final_cnn_labels = my_cnn_arh.metrics_names
print(f'CNN model {final_cnn_labels[0]} \t\t= {round(my_cnn_arh_score[0], 3)}')
print(f'CNN model {final_cnn_labels[1]} \t= {round(my_cnn_arh_score[1], 3)}')

In [ ]:
my_cnn_arh.save(f"save/saved_model_acc_{round(my_cnn_arh_score[1], 3)}.h5", save_format='h5')

In [ ]:
from keras.models import load_model
from deepface import DeepFace
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 
import cv2
import numpy as np

model = load_model("save/final_cnn_model_acc_0.575.h5")
cap = cv2.VideoCapture(0)
while True:
    ret, img = cap.read()
    face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    faces = face_cascade.detectMultiScale(img, scaleFactor=1.2, minNeighbors=6, minSize=(100, 100))

    for i, (x, y, w, h) in enumerate(faces):
        
        img2 = img[y:y+h, x:x+w]
        img_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
        img_gray = cv2.resize(img_gray, (200, 200))
        img_gray = img_gray.reshape(-1, 200, 200, 1)

        prediction = model.predict(img_gray)
        # age_ranges = ['1-3', '4-8', '9-13', '14-17', '18-21', '22-24', '25-27', '28-30', '31-35', '36-40', '41-45', '46-50']
        age_ranges = ['1-8', '9-17', '18-24', '25-30', '31-40', '41-50']
        v = np.argmax(prediction[0])

        face_rect = cv2.rectangle(img, (x, y), (x+w, y+h), (0, 100, 0), thickness=2)
        face_rect = cv2.rectangle(img, (x-1, y), (x+w+1, y-50),(0, 100, 0),thickness=-1)
        
        ag_text = str(age_ranges[v])+' лет'
        ag_text2 ='('+str(prediction[0][v])+')'
        
        font = cv2.FONT_HERSHEY_COMPLEX
        pos = (x,y-25)
        pos2 = (x,y-5)
        fontScale = 0.6
        fontColor = (255,255,255)
        thickness = 1
        lineType = 2
        
        cv2.putText(img,ag_text, 
            pos, 
            font, 
            fontScale,
            fontColor,
            thickness,
            lineType)
        
        cv2.putText(img,ag_text2, 
            pos2, 
            font, 
            fontScale,
            fontColor,
            thickness,
            lineType)
        
    cv2.imshow("camera", img)
    if cv2.waitKey(10) == 27: # Клавиша Esc
        break

cap.release()
cv2.destroyAllWindows()
    

In [ ]:
from keras.models import load_model
from deepface import DeepFace
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 
import cv2
from mtcnn import MTCNN
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread("test/tests.jpg")  
model = load_model("save/weight.h5")

detector = MTCNN() # создание детектора
result = detector.detect_faces(img)

# for i in range(len(result)): 
#     bounding_box = result[i]['box']  #  ограничивающие рамки
#     cv2.rectangle(img, (bounding_box[0], bounding_box[1]),
#     (bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3]),
#            (0,155,255), 2) #  ограничение области на изображении
#     plt.imshow(image)
# plt.imshow(image)
# face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
# faces = face_cascade.detectMultiScale(img, scaleFactor=1.2, minNeighbors=6, minSize=(100, 100))
for i in range(len(result)):
        bounding_box = result[i]['box']
        img2 = img[bounding_box[1]:bounding_box[1]+bounding_box[3], bounding_box[0]:bounding_box[0]+bounding_box[2]]
        img_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
        img_gray = cv2.resize(img_gray, (200, 200))
        img_gray = img_gray.reshape(-1, 200, 200, 1)

        prediction = model.predict(img_gray)
        # age_ranges = ['1-3', '4-8', '9-13', '14-17', '18-21', '22-24', '25-27', '28-30', '31-35', '36-40', '41-45', '46-50']
        age_ranges = ['1-8', '9-17', '18-24', '25-30', '31-40', '41-50']
        v = np.argmax(prediction[0])

        cv2.rectangle(img, (bounding_box[0], bounding_box[1]), (bounding_box[0]+bounding_box[2], bounding_box[1]+bounding_box[3]), (0, 100, 0), 2)
        cv2.rectangle(img, (bounding_box[0]- 1, bounding_box[1]), (bounding_box[0]+bounding_box[2]+1, bounding_box[1]-50),(0, 100, 0),-1)
        
        ag_text = str(age_ranges[v])+' лет'
        # ag_text2 ='('+str(prediction[0][v])+')'
        
        font = cv2.FONT_HERSHEY_COMPLEX
        pos = (bounding_box[0],bounding_box[1]-25)
        pos2 = (bounding_box[0],bounding_box[1]-5)
        fontScale = 0.6
        fontColor = (255,255,255)
        thickness = 1
        lineType = 2
        
        cv2.putText(img,ag_text, 
            pos, 
            font, 
            fontScale,
            fontColor,
            thickness,
            lineType)
        
        # cv2.putText(img,ag_text2, 
        #     pos2, 
        #     font, 
        #     fontScale,
        #     fontColor,
        #     thickness,
        #     lineType)
        
cv2.imshow("fhoto", img)
cv2.waitKey(55000)
cv2.destroyAllWindows()

    

In [ ]:
my_cnn_arh_predict = my_cnn_arh.predict(test_dataset)
my_cnn_arh_predict = my_cnn_arh_predict.argmax(axis=-1)
conf_mat = confusion_matrix(test_ll, my_cnn_arh_predict)
conf_mat

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion Matrix', export_as='confusion_matrix', cmap=plt.cm.Reds):

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=16)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Реальный возраст', fontsize=14)
    plt.xlabel('Предсказанный возраст', fontsize=14)

cm_plot_labels = ['1-8', '9-17', '18-24', '25-30', '31-40', '41-50']

plt.figure(figsize=(16,8))
plot_confusion_matrix(conf_mat, cm_plot_labels, normalize=False,title="Матрица ошибок основанная на предсказаниях модели CNN",export_as="final_cnn_conf_mat")
plt.show()

In [1]:
from keras.models import load_model
from deepface import DeepFace
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw
from mtcnn import MTCNN
import numpy as np
import matplotlib.pyplot as plt
from tkinter import *
import tkinter.filedialog as fd
import cv2
from PIL import Image,ImageTk

root = Tk()
fix_h = 550
kh = 0
model = load_model("save/weight.h5")
def choose_file():
    
        filetypes = (("Изображение", "*.jpg *.gif *.png"),
                     ("Любой", "*"))
        filename = fd.askopenfilename(title="Открыть файл", initialdir="/",
                                      filetypes=filetypes)
        if filename:
            
            for widget in frame2.winfo_children():
                widget.destroy()
            for widget in frame3.winfo_children():
                widget.destroy()
                
            img = cv2.imread(filename)  

            detector = MTCNN()
            result = detector.detect_faces(img)
            text_age = ""
            
            if result:
                for i in range(len(result)):

                    bounding_box = result[i]['box']
                    img2 = img[bounding_box[1]:bounding_box[1]+bounding_box[3], bounding_box[0]:bounding_box[0]+bounding_box[2]]
                    img_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
                    img_gray = cv2.resize(img_gray, (200, 200))
                    img_gray = img_gray.reshape(-1, 200, 200, 1)

                    prediction = model.predict(img_gray)
                    age_ranges = ['1-8', '9-17', '18-24', '25-30', '31-40', '41-50']
                    v = np.argmax(prediction[0])

                    cv2.rectangle(img, (bounding_box[0], bounding_box[1]), (bounding_box[0]+bounding_box[2], bounding_box[1]+bounding_box[3]), (0, 100, 0), 2)
                    cv2.rectangle(img, (bounding_box[0]- 1, bounding_box[1]), (bounding_box[0]+bounding_box[2]+1, bounding_box[1]-50),(0, 100, 0),-1)

                    ag_text = str(i+1)+': '+str(age_ranges[v])

                    font = cv2.FONT_HERSHEY_COMPLEX
                    pos = (bounding_box[0],bounding_box[1]-25)
                    pos2 = (bounding_box[0],bounding_box[1]-5)
                    fontScale = 0.8
                    fontColor = (255,255,255)
                    thickness = 1
                    lineType = 2

                    text_age += "Возраст "+str(i+1)+" человека "+str(age_ranges[v])+' лет.'

                    cv2.putText(img,ag_text, 
                        pos, 
                        font, 
                        fontScale,
                        fontColor,
                        thickness,
                        lineType)

                b,g,r = cv2.split(img)
                imgr = cv2.merge((r,g,b))
                im = Image.fromarray(imgr)

                height_percent = (fix_h / float(im.size[1]))
                width_size = int((float(im.size[0]) * float(height_percent)))
                new_image = im.resize((width_size, fix_h))
                imaget = ImageTk.PhotoImage(new_image)

                lb = Label(frame2,
                            height =fix_h,
                            image=imaget)
                lb.imaget = imaget
                lb.pack(padx=30)

                lb2 = Label(frame3,
                            font="Arial 22",
                            wraplength=1000,
                            justify=LEFT,
                            foreground="#ffffff",
                            background="#767676",
                            text="На фото было найдено "+str(len(result))+" человек(а).\n"+text_age,
                            )
                lb2.pack(side=LEFT, anchor=N, )
            else:
                
                b,g,r = cv2.split(img)
                imgr = cv2.merge((r,g,b))
                im = Image.fromarray(imgr)

                height_percent = (fix_h / float(im.size[1]))
                width_size = int((float(im.size[0]) * float(height_percent)))
                new_image = im.resize((width_size, fix_h))
                imaget = ImageTk.PhotoImage(new_image)
                
                lb = Label(frame2,
                            height =fix_h,
                            image=imaget)
                lb.imaget = imaget
                lb.pack(padx=30)
                
                lb2 = Label(frame3,
                            font="Arial 22",
                            wraplength=1000,
                            justify=LEFT,
                            foreground="#ffffff",
                            background="#767676",
                            text="Не было найдено людей на фото!"
                            )
                lb2.pack(side=LEFT, anchor=N, )
               
                
root['bg'] = '#fafafa'
root.title('Приложение для определения возраста')
root.geometry('1200x700')

frame1 = Frame(root,bg='#989898',bd=5)
frame1.place(relx=0,rely=0,relwidth=0.20,relheight=1)

frame2 = Frame(root,bg='#555555',bd=5)
frame2.place(relx=0.20,rely=0,relwidth=0.80,relheight=1)

frame3 = Frame(root,bg='#767676',bd=5)
frame3.place(relx=0.20,rely=0.8,relwidth=0.80,relheight=1)

btn = Button(frame1,
             text='Выбрать фото',
             width = 500,
             height = 3,
             background="#555",
             foreground="#ffffff",
             command=choose_file)
btn.pack(padx=10, pady=10)
root.mainloop()
